In [1]:
import pandas as pd, numpy as np, re
from pathlib import Path

In [10]:
working_dir = Path('/d/data/plasmo/nn_scalar3')
holdout_file = working_dir / 'output_mod/predicted_genes_usethis.tsv'
# v3_file = working_dir / 'output_v3/predicted_genes.tsv'
# v4_file = working_dir / 'output_v4/predicted_genes_v4.tsv'
fastlmm_file = Path('/d/data/plasmo/fastlmm/fastlmm_top100.txt')
pub_file = Path('/d/data/plasmo/fastlmm/pub_results.txt')

In [11]:
# v3_df = pd.read_csv(v3_file, sep='\t')
# v4_df = pd.read_csv(v4_file, sep='\t')
fastlmm_df = pd.read_csv(fastlmm_file, sep='\t')
pub_df = pd.read_csv(pub_file, sep='\t')
holdout_df = pd.read_csv(holdout_file, sep='\t')

In [12]:
#take the position ids and translate them to block
def toBlock(name):
    sl = 5000.
    chr, pos = name.split(':')
    block = int(np.floor(int(pos) / sl) * sl)
    return '{0}:{1}'.format(chr, str(block))
pub_df['block_id'] = pub_df['id'].apply(toBlock)
fastlmm_df['block_id'] = fastlmm_df['id'].apply(toBlock)

In [13]:
#figure out the intersections
# v3_hits = v3_df.loc[:,'id']
# v4_hits = v4_df.loc[:, 'id']
holdout_hits = holdout_df.loc[:,'id']
pub_hits = pub_df.loc[:,'block_id']
fastlmm_hits = fastlmm_df.loc[:, 'block_id']

In [14]:
from functools import reduce
intersect_all = reduce(lambda x, y: np.intersect1d(x, y), [pub_hits, holdout_hits])

In [15]:
print(intersect_all)

['Pf3D7_13_v3:1725000']


In [16]:
print(np.intersect1d(holdout_hits, fastlmm_hits))

['Pf3D7_13_v3:1695000' 'Pf3D7_13_v3:1725000']


In [ ]:
intersect_mine = reduce(lambda x, y: np.intersect1d(x, y), [holdout_hits, fastlmm_hits])
print(intersect_mine)

In [ ]:
my_hits = holdout_df.set_index('id').loc[intersect_mine, 'genes']
print(my_hits)

In [ ]:
agreed_hits = holdout_df.set_index('id').loc[intersect_all, 'genes']
print(agreed_hits)

In [ ]:
diffs = intersect_mine[np.invert(np.in1d(intersect_mine, intersect_all))]
print(diffs)
novel_hits = holdout_df.set_index('id').loc[diffs, :]
print(novel_hits)